# Dataset

In [1]:
import pandas as pd
import numpy as np
import json

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import altair as alt
alt.renderers.enable('notebook')
alt.data_transformers.enable('default', max_rows=None)

from utils import *
%load_ext autoreload
%autoreload 2

In [4]:
df = load_json(path='exponea_prediction_dummy_data/campaign.json')
df = rem_const(df)
# for easier exploration
df.drop(['recipient','language'],1,inplace=True)

In [5]:
# timerange
[df.iloc[0,:].date,df.iloc[-1,:].date]

[Timestamp('2018-03-17 07:05:28.764564037'),
 Timestamp('2018-05-16 06:50:46.329377890')]

## Dataset creation

In [6]:
# https://knowledge.hubspot.com/email-user-guide-v2/understanding-email-deliverability
# query just emails since opened/clicked except for 2 instances (pushweb) are only emails
# ie action_type='split' => status='failed' are out in that case

strng = 'email'
df = df.query("(action_type == @strng)")
df.drop('action_type',1,inplace=True)

In [8]:
# other statuses shouldn't matter

strng = "delivered"
strng1 = 'opened'
strng2 = 'clicked'
sample = df.query("status == @strng | status == @strng1 | status == @strng2")

In [9]:
# create unique identifier for customer + campaign
sample['unique_id'] = sample['customer_id'].map(str) + sample['campaign_id']

In [10]:
# need to drop duplated rows
sample.drop_duplicates(inplace=True)

In [11]:
# drop clicked to unsubscribe since this is not positive response
sample = sample[~sample['url'].str.contains("unsubscribe",na=False)]

In [12]:
# prepare df for loop to be in correct order

sample = sample.sort_values(by=['unique_id','date'])
sample.reset_index(inplace=True)

In [13]:
# tryng to iterate over rows and look if in sorted df
# row below is success given current row is delivered
# if not we consider unsuccess
output = []
val = None
for i, row in sample.iterrows():
    #print(i)
    #print(sample.loc[i,:])
    # only delivered can be opened/clicked
    if (row.status == 'delivered') & (i < len(sample)-1):
        #since we sorted by id & date we row below has to be opened/clicked
        # to be success
        if (sample.loc[i+1,:].status != 'delivered') & (row.unique_id == sample.loc[i+1,:].unique_id):
            val = 1
        else:
            val = 0
    else:
        val = 0
    output.append(val)     

In [14]:
# assign response variable
sample['response'] = output
sample['hour'] = sample.date.dt.hour

In [16]:
# take just training instances
strng = 'delivered'
df = sample.query('status == @strng')

## Create new features

##### Purchased item

In [19]:
# this is more of pipeline for future predictions

In [170]:
purchased = load_json(path='exponea_prediction_dummy_data/purchase_item.json')
prch = rem_const(purchased)
prch['freq'] = prch.groupby('customer_id')['customer_id'].transform('count')
prch['last'] = prch.groupby('customer_id')['date'].transform('last')
prch.sort_values(by=['customer_id','date'], inplace=True)

In [172]:
# mean difference in minutes between puchases for each customer
prch['diff'] = prch.groupby(['customer_id'])['date'].transform(lambda x: x.diff().dt.seconds.div(60))
prch['mean_diff'] = prch.groupby('customer_id')['diff'].transform('mean').fillna(0)
prch.drop('diff', 1,inplace=True)

##### Cart update

In [24]:
cart = load_json(path='exponea_prediction_dummy_data/cart_update.json')
cart = rem_const(cart)
cart['freq'] = cart.groupby('customer_id')['customer_id'].transform('count')
cart['last'] = cart.groupby('customer_id')['date'].transform('last')
cart.sort_values(by=['customer_id','date'], inplace=True)

In [25]:
# mean difference in minutes between cart updates for each customer
cart['diff'] = cart.groupby(['customer_id'])['date'].transform(lambda x: x.diff().dt.seconds.div(60))
cart['mean_diff'] = cart.groupby('customer_id')['diff'].transform('mean').fillna(0)
cart.drop('diff', 1,inplace=True)

#### Email openings metrics

In [33]:
# how many unique campaign customer opened and with what mean differences

In [19]:
strng = 'clicked'
strng1 = 'opened'
sample = sample.query('status == @strng | status ==@strng1')
sample.sort_values(by='date',inplace=True)
email = sample[~sample.duplicated(subset='unique_id')]

In [ ]:
email['freq'] = email.groupby('customer_id')['customer_id'].transform('count')
email['last'] = email.groupby('customer_id')['date'].transform('last')
email.sort_values(by=['customer_id','date'], inplace=True)

In [ ]:
# mean difference in minutes between oepened/clicked emails for each customer 
email['diff'] = email.groupby(['customer_id'])['date'].transform(lambda x: x.diff().dt.seconds.div(60))
email['mean_diff'] = email.groupby('customer_id')['diff'].transform('mean').fillna(0)
email.drop('diff', 1,inplace=True)

## Mapping sets of features to each row

In [33]:
# take delivered email.date load datatsets and recalculate
# for corner cases we will use Naive population model

In [24]:
# prch
purchased = load_json(path='exponea_prediction_dummy_data/purchase_item.json')
prch = rem_const(purchased)

# cart
cart = load_json(path='exponea_prediction_dummy_data/cart_update.json')
cart = rem_const(cart)

# view
view = load_json(path='exponea_prediction_dummy_data/view_product.json')
view.drop('eans',1,inplace=True)
view = rem_const(view)

# session_start
sess = load_json(path='exponea_prediction_dummy_data/session_start.json')
sess = rem_const(sess)

In [ ]:
# Below takes unreasonably long

prch_feats = []
cart_feats = []
view_feats = []
sess_feats = []
email_feats = []


import time
start = time.time()

for i,row in df.iterrows():
    print(i)
    # for each row create subset that was actual in time of the email
    temp1 = prch[(prch.date < row.date) & (prch.customer_id == row.customer_id)]
    prch_feats.append(add_metrics(temp1))
    
    temp2 = cart[(cart.date < row.date) & (cart.customer_id == row.customer_id)]
    cart_feats.append(add_metrics(temp2))
    
    temp3 = view[(view.date < row.date) & (view.customer_id == row.customer_id)]
    view_feats.append(add_metrics(temp3))
    
    temp4 = sess[(sess.date < row.date) & (sess.customer_id == row.customer_id)]
    sess_feats.append(add_metrics(temp4))
    
    temp5 = email[(email.date < row.date) & (email.customer_id == row.customer_id)]
    email_feats.append(add_metrics(temp5))
    
print(time.time-start)

In [ ]:
a = prch_feats
b = cart_feats
c = view_feats
d = sess_feats
e = email_feats

In [ ]:
# in seconds but probably will need to be updated
dataset['prch_freq'] = list(zip(*a))[0]
dataset['prch_last'] = (dataset.date - pd.to_datetime(list(zip(*a))[1],errors='coerce')).dt.seconds.fillna(0).div(60)
dataset['prch_mean_diff'] = list(zip(*a))[2]

dataset['cart_freq'] = list(zip(*b))[0]
dataset['cart_last'] = (dataset.date - pd.to_datetime(list(zip(*b))[1],errors='coerce')).dt.seconds.fillna(0).div(60)
dataset['cart_mean_diff'] = list(zip(*b))[2]


dataset['view_freq'] = list(zip(*c))[0]
dataset['view_last'] = (dataset.date - pd.to_datetime(list(zip(*c))[1],errors='coerce')).dt.seconds.fillna(0).div(60)
dataset['view_mean_diff'] = list(zip(*c))[2]


dataset['sess_freq'] = list(zip(*d))[0]
dataset['sess_last'] = (dataset.date - pd.to_datetime(list(zip(*d))[1],errors='coerce')).dt.seconds.fillna(0).div(60)
dataset['sess_mean_diff'] = list(zip(*d))[2]


dataset['email_freq'] = list(zip(*e))[0]
dataset['email_last'] = (dataset.date - pd.to_datetime(list(zip(*e))[1],errors='coerce')).dt.seconds.fillna(0).div(60)
dataset['email_mean_diff'] = list(zip(*e))[2]

In [53]:
dataset.to_csv('df.csv')